SQL Miscellanea
-----

In [1]:
%load_ext sql
%sql sqlite:///world-db

u'Connected: @world-db'

We can create views in SQL. Views are not stored as tables (materialized), but they can be used inside other SQL queries as normal tables.

In [7]:
%%sql
DROP VIEW OfficialCountryLanguage;

CREATE VIEW OfficialCountryLanguage AS
SELECT C.Name AS CountryName, L.Language AS Language
FROM CountryLanguage L, Country C
WHERE L.CountryCode = C.Code
AND L.IsOfficial = 'T' ;

 * sqlite:///world-db
Done.
Done.


[]

In [8]:
%sql SELECT * FROM OfficialCountryLanguage LIMIT 10;

 * sqlite:///world-db
(sqlite3.OperationalError) no such table: main.CountryLanguage [SQL: u'SELECT * FROM OfficialCountryLanguage LIMIT 10;'] (Background on this error at: http://sqlalche.me/e/e3q8)


The `WITH` clause can be added before the `SELECT` statement to define a table that can be used in the main query statement.

In [9]:
%%sql
WITH C AS (SELECT Name, Continent
      FROM Country
      WHERE Population >50000000)
SELECT C.Name
FROM C
WHERE C.Continent = 'Europe' ;

 * sqlite:///world-db
(sqlite3.OperationalError) no such table: Country [SQL: u"WITH C AS (SELECT Name, Continent\n      FROM Country\n      WHERE Population >50000000)\nSELECT C.Name\nFROM C\nWHERE C.Continent = 'Europe' ;"] (Background on this error at: http://sqlalche.me/e/e3q8)


**MEDIAN**

In the following, we will show how one can compute the *median* of a table column using SQL. Recall that to compute the median of a (multi)set of *n* values, we first order the values; then, we return the middle number if *n* is odd, otherwise the average of the two middle numbers. 

We define the following table for our case:

In [10]:
%%sql
DROP TABLE IF EXISTS R;
CREATE TABLE R(A integer);
INSERT INTO R Values (1),(5),(9),(3),(10),(10),(10),(25);

 * sqlite:///world-db
Done.
Done.
8 rows affected.


[]

We will first solve a simplified version of the median problem. Let's for now assume that:
* the number of elements is odd
* there are no duplicate values

In [11]:
%%sql
SELECT X.A
FROM R AS X
WHERE (SELECT COUNT(*) FROM R AS X1 WHERE X.A > X1.A) 
= (SELECT COUNT(*) FROM R AS X2 WHERE X.A < X2.A);

 * sqlite:///world-db
Done.


A


The above solution will not work if our two assumptions do not hold (why?). Let us rewrite the query so that we can solve the general median problem.

In [12]:
%%sql
SELECT AVG(DISTINCT X.A)
FROM R AS X
WHERE (SELECT COUNT(*) FROM R AS X1 WHERE X.A >= X1.A) 
>= (SELECT COUNT(*) FROM R AS X2 WHERE X.A < X2.A)
AND (SELECT COUNT(*) FROM R AS X1 WHERE X.A > X1.A) 
<= (SELECT COUNT(*) FROM R AS X2 WHERE X.A <= X2.A);

 * sqlite:///world-db
Done.


AVG(DISTINCT X.A)
9.5


** Paths in Graphs**

We will next show how we can compute some queries on a graph. Here, we represent the graph a single relation with schema `Edge(source, target, distance)`.

In [13]:
%%sql
DROP TABLE IF EXISTS Edge;
CREATE TABLE Edge (source integer, target integer, distance integer);
INSERT INTO Edge VALUES (1,2,10),(2,3,10),(3,4,20),(4,1,30),(1,3,5); 

 * sqlite:///world-db
Done.
Done.
5 rows affected.


[]

The query below computes the number of outgoing edges for each vertex.

In [16]:
%%sql
SELECT source, COUNT(target)
FROM Edge
GROUP BY source;

 * sqlite:///world-db
Done.


source,COUNT(target)
1,2
2,1
3,1
4,1


Next, we want to find all the directed paths of length 2 in the graph.

In [17]:
%%sql
SELECT e1.source, e1.target, e2.target
FROM Edge e1, Edge e2
WHERE e1.target = e2.source; 

 * sqlite:///world-db
Done.


source,target,target_1
1,2,3
2,3,4
3,4,1
4,1,2
4,1,3
1,3,4


Can we also compute the distance of each path?

In [18]:
%%sql
SELECT e1.source, e1.target, e2.target, (e1.distance+e2.distance)
FROM Edge e1, Edge e2
WHERE e1.target = e2.source; 

 * sqlite:///world-db
Done.


source,target,target_1,(e1.distance+e2.distance)
1,2,3,20
2,3,4,30
3,4,1,50
4,1,2,40
4,1,3,35
1,3,4,25


How do we find all (directed) triangles in the graph?

In [19]:
%%sql
SELECT e1.source, e2.source, e3.source
FROM Edge e1, Edge e2, Edge e3
WHERE e1.target = e2.source AND e2.target=e3.source AND e3.target = e1.source; 

 * sqlite:///world-db
Done.


source,source_1,source_2
3,4,1
4,1,3
1,3,4


** A few things on recursion**

SQL allows recursion in the `WITH` clause.

In [20]:
%%sql
WITH RECURSIVE
  cnt(x) AS (VALUES(1) UNION SELECT x+1 FROM cnt WHERE x<10)
SELECT x FROM cnt;

 * sqlite:///world-db
Done.


x
1
2
3
4
5
6
7
8
9
10


In [25]:
%%sql
WITH RECURSIVE
    factorial(n,x) AS (
        SELECT 1, 1
        UNION
        SELECT n+1, (n+1)*x FROM factorial WHERE n < 5)
SELECT * FROM factorial ;

 * sqlite:///world-db
Done.


n,x
1,1
2,2
3,6
4,24
5,120
